In [44]:
!pip install google-genai

In [45]:
import os
import google.generativeai as genai

os.environ['GOOGLE_API_KEY'] =userdata.get('GOOGLE_API_KEY')

In [46]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

model = genai.GenerativeModel('gemini-2.0-flash')

print("Biblioteca Gemini configurada usando a variável de ambiente!")

Biblioteca Gemini configurada usando a variável de ambiente!


In [47]:
def analisar_vt_e_sugerir_musica(descricao_vt):
    prompt = f"""Analise a seguinte descrição de um vídeo (VT) e identifique o sentimento predominante,
o tom geral, o estilo e tente associar com arquétipos ou temas universais presentes.

Com base nessa análise, sugira alguns artistas ou músicas específicas que poderiam ser adequados para este VT.
Justifique brevemente cada sugestão com base na sua análise.

Descrição do VT:
{descricao_vt}
"""
    response = model.generate_content(prompt)
    return response.text

In [48]:
descricao_exemplo = "Um vídeo curto e divertido mostrando um grupo de amigos rindo e se divertindo em uma festa na praia ao pôr do sol. A música deve ser animada, alegre e ter uma vibe relaxante de verão."

resultado_gemini = analisar_vt_e_sugerir_musica(descricao_exemplo)
print(resultado_gemini)

## Análise da Descrição do VT

**Sentimento Predominante:** Alegria, felicidade, descontração e amizade.

**Tom Geral:** Leve, divertido, positivo e despreocupado.

**Estilo:** Casual, autêntico, vibrante e "instagramável" (visualmente atraente e adequado para redes sociais).

**Arquétipos/Temas Universais:**

*   **A Busca pela Felicidade:** A celebração da alegria e da experiência positiva.
*   **Camaradagem/Amizade:** O valor das conexões e momentos compartilhados com amigos.
*   **A Juventude:** A energia, vitalidade e a despreocupação típicas dessa fase da vida.
*   **Paraíso/Escape:** A praia ao pôr do sol como um cenário idílico de fuga da rotina.

## Sugestões de Artistas e Músicas

Com base na análise acima, aqui estão algumas sugestões de artistas e músicas que poderiam ser adequadas para este VT, juntamente com suas justificativas:

**1. Artista: Jack Johnson**

*   **Música Sugerida:** "Better Together" ou "Banana Pancakes"
*   **Justificativa:** Jack Johnson personifica a 

In [49]:
!pip install spotipy

In [50]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from google.colab import userdata

# Acesse as credenciais do Spotify dos segredos do Colab
SPOTIPY_CLIENT_ID = userdata.get('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = userdata.get('SPOTIPY_CLIENT_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                       client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

print("API do Spotify autenticada com sucesso!")

API do Spotify autenticada com sucesso!


In [51]:
def analisar_vt_e_sugerir_musica_com_spotify_e_playlists(descricao_vt):
    """
    Analisa a descrição de um VT usando o Gemini e sugere artistas/músicas e playlists
    usando também a API do Spotify.

    Args:
        descricao_vt: Uma string contendo a descrição textual detalhada do VT.

    Returns:
        Um dicionário contendo as sugestões de artistas/músicas e playlists
        geradas pelo Gemini e pela API do Spotify.
    """
    prompt_gemini_generos_simples = f"""Analise a seguinte descrição de um vídeo (VT) e liste até 3 gêneros musicais amplos que seriam adequados para a trilha sonora.
Liste apenas o nome dos gêneros, um por linha.

Descrição do VT:
{descricao_vt}
"""

    generos_sugeridos = []

    try:
        response_gemini_generos = model.generate_content(prompt_gemini_generos_simples)
        # Limpando os gêneros removendo caracteres extras
        generos_sugeridos = [genero.strip().lstrip('*').strip() for genero in response_gemini_generos.text.split('\n') if genero.strip()]
    except Exception as e:
        print(f"Erro ao obter gêneros do Gemini: {e}")

    sugestoes_spotify_musicas = []
    if sp and generos_sugeridos:
        for genero in generos_sugeridos:
            try:
                # Buscar por músicas (mantendo a lógica existente)
                results_tracks = sp.search(q=f'genre:{genero}', type='track', limit=3)
                if results_tracks is not None and 'tracks' in results_tracks and 'items' in results_tracks['tracks']:
                    for track in results_tracks['tracks']['items']:
                        sugestoes_spotify_musicas.append({
                            'artista': track['artists'][0]['name'],
                            'titulo': track['name'],
                            'album': track['album']['name'],
                            'url': track['external_urls']['spotify']
                        })
                else:
                    print(f"Aviso ao buscar músicas para o gênero '{genero}': Nenhum resultado válido retornado.")
            except Exception as e:
                print(f"Erro ao buscar músicas no Spotify para o gênero '{genero}': {e}")

    # Chamar a função para buscar playlists
    sugestoes_spotify_playlists = buscar_playlists_spotify(generos_sugeridos, sp)

    return {
        'generos_sugeridos_gemini': generos_sugeridos,
        'sugestoes_spotify_musicas': sugestoes_spotify_musicas,
        'sugestoes_spotify_playlists': sugestoes_spotify_playlists
    }

print("Função principal de análise e sugestão (limpeza de gêneros) criada!")

Função principal de análise e sugestão (limpeza de gêneros) criada!


In [52]:
def buscar_playlists_spotify(generos, sp_client):
    """
    Busca playlists no Spotify com base em uma lista de gêneros.

    Args:
        generos: Uma lista de strings contendo os gêneros musicais.
        sp_client: O objeto cliente da API do Spotify (spotipy.Spotify).

    Returns:
        Uma lista de dicionários, onde cada dicionário representa uma playlist
        encontrada (nome, proprietário, url). Retorna uma lista vazia em caso de erro
        ou se nenhum resultado for encontrado.
    """
    sugestoes_spotify_playlists = []
    if sp_client and generos:
        for genero in generos:
            try:
                results_playlists = sp_client.search(q=f'genre:{genero}', type='playlist', limit=2)
                if results_playlists is not None and 'playlists' in results_playlists and isinstance(results_playlists['playlists'], dict) and 'items' in results_playlists['playlists']:
                    try:
                        for playlist in results_playlists['playlists']['items']:
                            sugestoes_spotify_playlists.append({
                                'nome': playlist['name'],
                                'proprietario': playlist['owner']['display_name'],
                                'url': playlist['external_urls']['spotify']
                            })
                    except TypeError:
                        print(f"Aviso: Erro ao iterar sobre os itens de playlist para o gênero '{genero}'. A resposta 'items' pode ser None.")
                elif results_playlists is not None and 'playlists' in results_playlists:
                    print(f"Aviso: Estrutura de playlist inesperada para o gênero '{genero}'. Chaves: {results_playlists['playlists'].keys()}")
                elif results_playlists is not None:
                    print(f"Aviso: Resposta de playlist inesperada para o gênero '{genero}'. Chaves: {results_playlists.keys()}")
                else:
                    print(f"Aviso: Nenhum resultado de playlist válido retornado para o gênero '{genero}'.")

            except Exception as e:
                print(f"Erro ao buscar playlists no Spotify para o gênero '{genero}': {e}")
    return sugestoes_spotify_playlists

print("Função para buscar playlists no Spotify criada!")

Função para buscar playlists no Spotify criada!


In [53]:
def buscar_mais_musicas(genero, sp_client, limit=5):
    """Busca mais músicas no Spotify para um gênero específico."""
    sugestoes_adicionais = []
    if sp_client and genero:
        try:
            results_tracks = sp_client.search(q=f'genre:{genero}', type='track', limit=limit)
            if results_tracks is not None and 'tracks' in results_tracks and 'items' in results_tracks['tracks']:
                for track in results_tracks['tracks']['items']:
                    sugestoes_adicionais.append({
                        'artista': track['artists'][0]['name'],
                        'titulo': track['name'],
                        'album': track['album']['name'],
                        'url': track['external_urls']['spotify']
                    })
            else:
                print(f"Aviso: Nenhum resultado adicional de música encontrado para o gênero '{genero}'.")
        except Exception as e:
            print(f"Erro ao buscar mais músicas para o gênero '{genero}': {e}")
    return sugestoes_adicionais

def buscar_mais_playlists(genero, sp_client, limit=3):
    """Busca mais playlists no Spotify para um gênero específico."""
    sugestoes_adicionais = []
    if sp_client and genero:
        try:
            results_playlists = sp_client.search(q=f'genre:{genero}', type='playlist', limit=limit)
            if results_playlists is not None and 'playlists' in results_playlists and 'items' in results_playlists['playlists']:
                for playlist in results_playlists['playlists']['items']:
                    sugestoes_adicionais.append({
                        'nome': playlist['name'],
                        'proprietario': playlist['owner']['display_name'],
                        'url': playlist['external_urls']['spotify']
                    })
            else:
                print(f"Aviso: Nenhum resultado adicional de playlist encontrado para o gênero '{genero}'.")
        except Exception as e:
            print(f"Erro ao buscar mais playlists para o gênero '{genero}': {e}")
    return sugestoes_adicionais

offset_artistas = 0 # Variável global para controlar o offset dos artistas

def buscar_artistas_relacionados(genero, sp_client, limit=5, offset=0):
    """Busca artistas relacionados a um gênero específico com um offset."""
    global offset_artistas
    artistas_relacionados = []
    if sp_client and genero:
        try:
            results_artists = sp_client.search(q=f'genre:{genero}', type='artist', limit=limit, offset=offset)
            if results_artists is not None and 'artists' in results_artists and 'items' in results_artists['artists']:
                for artist in results_artists['artists']['items']:
                    artistas_relacionados.append({
                        'nome': artist['name'],
                        'url': artist['external_urls']['spotify']
                    })
                offset_artistas += limit # Incrementa o offset para a próxima busca
            else:
                print(f"Aviso: Nenhum artista relacionado encontrado para o gênero '{genero}' (offset: {offset}).")
        except Exception as e:
            print(f"Erro ao buscar artistas relacionados para o gênero '{genero}' (offset: {offset}): {e}")
    return artistas_relacionados

print("Função buscar_artistas_relacionados modificada para loop!")

Função buscar_artistas_relacionados modificada para loop!


In [54]:
offset_musicas = 0 # Variável global para controlar o offset das músicas

def buscar_mais_musicas_relacionadas(genero, sp_client, limit=5, offset=0):
    """Busca mais músicas relacionadas a um gênero específico com um offset."""
    global offset_musicas
    sugestoes_adicionais = []
    if sp_client and genero:
        try:
            results_tracks = sp_client.search(q=f'genre:{genero}', type='track', limit=limit, offset=offset)
            if results_tracks is not None and 'tracks' in results_tracks and 'items' in results_tracks['tracks']:
                for track in results_tracks['tracks']['items']:
                    sugestoes_adicionais.append({
                        'artista': track['artists'][0]['name'],
                        'titulo': track['name'],
                        'album': track['album']['name'],
                        'url': track['external_urls']['spotify']
                    })
                offset_musicas += limit # Incrementa o offset para a próxima busca
            else:
                print(f"Aviso: Nenhum resultado adicional de música encontrado para o gênero '{genero}' (offset: {offset}).")
        except Exception as e:
            print(f"Erro ao buscar mais músicas para o gênero '{genero}' (offset: {offset}): {e}")
    return sugestoes_adicionais

print("Função buscar_mais_musicas_relacionadas criada!")

Função buscar_mais_musicas_relacionadas criada!


In [55]:
from google.colab import files

def exportar_resultados(resultados_principais, resultados_musicas_refinadas, resultados_playlists_refinadas, resultados_artistas_refinados):
    """Exporta os resultados principais e refinados para um arquivo TXT."""
    nome_arquivo = "vt_music_suggestions.txt"

    with open(nome_arquivo, 'w') as f:
        f.write("--- Análise do Gemini ---\n")
        if 'generos_sugeridos_gemini' in resultados_principais:
            f.write(f"Gêneros Sugeridos: {', '.join(resultados_principais['generos_sugeridos_gemini'])}\n")
        f.write("\n--- Sugestões do Spotify (Músicas Originais) ---\n")
        if 'sugestoes_spotify_musicas' in resultados_principais:
            for musica in resultados_principais['sugestoes_spotify_musicas']:
                f.write(f"Artista: {musica['artista']}\n")
                f.write(f"Título: {musica['titulo']}\n")
                f.write(f"Álbum: {musica['album']}\n")
                f.write(f"Link: {musica['url']}\n\n")
        f.write("\n--- Sugestões do Spotify (Playlists Originais) ---\n")
        if 'sugestoes_spotify_playlists' in resultados_principais:
            for playlist in resultados_principais['sugestoes_spotify_playlists']:
                f.write(f"Nome: {playlist['nome']}\n")
                f.write(f"Proprietário: {playlist['proprietario']}\n")
                f.write(f"Link: {playlist['url']}\n\n")

        if resultados_musicas_refinadas:
            f.write("\n--- Sugestões Adicionais de Músicas (Refinamento) ---\n")
            for musica in resultados_musicas_refinadas:
                f.write(f"Artista: {musica['artista']}\n")
                f.write(f"Título: {musica['titulo']}\n")
                f.write(f"Álbum: {musica['album']}\n")
                f.write(f"Link: {musica['url']}\n\n")

        if resultados_playlists_refinadas:
            f.write("\n--- Sugestões Adicionais de Playlists (Refinamento) ---\n")
            for playlist in resultados_playlists_refinadas:
                f.write(f"Nome: {playlist['nome']}\n")
                f.write(f"Proprietário: {playlist['proprietario']}\n")
                f.write(f"Link: {playlist['url']}\n\n")

        if resultados_artistas_refinados:
            f.write("\n--- Artistas Relacionados (Refinamento) ---\n")
            for artista in resultados_artistas_refinados:
                f.write(f"Nome: {artista['nome']}\n")
                f.write(f"Link: {artista['url']}\n\n")

    files.download(nome_arquivo)
    print(f"Resultados exportados para '{nome_arquivo}'")

In [59]:
def formatar_resultado_com_playlists(resultado):
    """Formata o resultado da análise para exibição."""
    output = "## Análise do Gemini:\n\n"
    if resultado['generos_sugeridos_gemini']:
        output += "* " + "\n* ".join(resultado['generos_sugeridos_gemini']) + "\n\n"
    else:
        output += "Nenhum gênero sugerido.\n\n"

    output += "## Sugestões de Músicas do Spotify:\n\n"
    if resultado['sugestoes_spotify_musicas']:
        for musica in resultado['sugestoes_spotify_musicas']:
            output += f"* **Artista:** {musica['artista']}\n"
            output += f"* **Título:** {musica['titulo']}\n"
            output += f"* **Álbum:** {musica['album']}\n"
            output += f"* **Link:** {musica['url']}\n\n"
    else:
        output += "Nenhuma música encontrada para os gêneros sugeridos.\n\n"

    output += "## Sugestões de Playlists do Spotify:\n\n"
    if resultado['sugestoes_spotify_playlists']:
        for playlist in resultado['sugestoes_spotify_playlists']:
            output += f"* **Nome:** {playlist['nome']}  \n"
            output += f"* **Proprietário:** {playlist['proprietario']}\n"
            output += f"* **Link:** {playlist['url']}\n\n"
    else:
        output += "Nenhuma playlist encontrada para os gêneros sugeridos.\n\n"

    return output

print("Função principal de análise e formatação criada!")

Função principal de análise e formatação criada!


In [ ]:
#@title Insira a descrição do seu vídeo aqui:
descricao_do_vt = "Um vídeo de skate com manobras radicais em diferentes cenários urbanos. Câmera lenta em alguns momentos para destacar os truques. A trilha sonora deve ser enérgica, com batidas fortes e um ritmo que impulsione a ação" #@param {type:"string"}

resultados_musicas_refinadas = []
resultados_playlists_refinadas = []
resultados_artistas_refinados = []

def refinar_busca(genero):
    global offset_musicas
    global offset_artistas
    global resultados_musicas_refinadas
    global resultados_playlists_refinadas
    global resultados_artistas_refinados
    offset_musicas = 0
    offset_artistas = 0
    while True:
        print(f"\nOpções para refinar a busca no gênero '{genero}':")
        print("1. Buscar mais músicas relacionadas")
        print("2. Buscar mais playlists relacionadas")
        print("3. Buscar mais artistas relacionados")
        print("4. Voltar ao menu principal")
        opcao = input("Digite o número da opção desejada: ")
        if opcao == '1':
            resultados = buscar_mais_musicas_relacionadas(genero, sp, offset=offset_musicas)
            if resultados:
                print("\nSugestões adicionais de músicas relacionadas:")
                for sugestao in resultados:
                    print(f"* Artista: {sugestao['artista']}, Título: {sugestao['titulo']}")
                    resultados_musicas_refinadas.append(sugestao) # Adiciona aos resultados refinados
                input("\nPressione Enter para ver mais músicas relacionadas...")
            else:
                print("Nenhuma sugestão adicional de música relacionada encontrada.")
        elif opcao == '2':
            resultados = buscar_mais_playlists(genero, sp)
            if resultados:
                print("\nSugestões adicionais de playlists relacionadas:")
                for sugestao in resultados:
                    print(f"* Nome: {sugestao['nome']}, Proprietário: {sugestao['proprietario']}")
                    resultados_playlists_refinadas.append(sugestao) # Adiciona aos resultados refinados
                input("\nPressione Enter para ver mais playlists relacionadas...")
            else:
                print("Nenhuma sugestão adicional de playlist relacionada encontrada.")
        elif opcao == '3':
            artistas = buscar_artistas_relacionados(genero, sp, offset=offset_artistas)
            if artistas:
                print("\nArtistas relacionados:")
                for artista in artistas:
                    print(f"* Nome: {artista['nome']}")
                    resultados_artistas_refinados.append(artista) # Adiciona aos resultados refinados
                input("\nPressione Enter para ver mais artistas relacionados...")
            else:
                print("Nenhum artista relacionado adicional encontrado.")
        elif opcao == '4':
            break
        else:
            print("Opção inválida. Por favor, digite um número de 1 a 4.")

if descricao_do_vt:
    resultados_principais = analisar_vt_e_sugerir_musica_com_spotify_e_playlists(descricao_do_vt)
    print(formatar_resultado_com_playlists(resultados_principais))

    while True:
        print("\nOpções:")
        print("1. Refinar busca por gênero")
        print("2. Sair e exportar resultados (TXT)")
        opcao_principal = input("Digite o número da opção desejada: ")

        if opcao_principal == '1':
            generos_sugeridos = resultados_principais['generos_sugeridos_gemini']
            if generos_sugeridos:
                print("\nVocê gostaria de refinar a busca por algum destes gêneros?")
                for i, genero in enumerate(generos_sugeridos):
                    print(f"{i+1}. {genero}")
                genero_escolhido_index = input("Digite o número do gênero para refinar (ou pressione Enter para continuar): ")
                if genero_escolhido_index.isdigit() and 1 <= int(genero_escolhido_index) <= len(generos_sugeridos):
                    genero_refinado = generos_sugeridos[int(genero_escolhido_index) - 1]
                    refinar_busca(genero_refinado)
                else:
                    print("\nNenhuma refinação de gênero selecionada.")
            else:
                print("Nenhum gênero sugerido para refinar.")
        elif opcao_principal == '2':
            exportar_resultados(resultados_principais, resultados_musicas_refinadas, resultados_playlists_refinadas, resultados_artistas_refinados)
            print("Saindo e exportando os resultados.")
            break
        else:
            print("Opção inválida. Por favor, digite 1 ou 2.")

else:
    print("Por favor, insira a descrição do vídeo no campo acima.")

## Análise do Gemini:

* Hip Hop
* Rock
* Eletrônica

## Sugestões de Músicas do Spotify:

* **Artista:** XXXTENTACION
* **Título:** Hope
* **Álbum:** ?
* **Link:** https://open.spotify.com/track/2ZRo7axmMPeSVUvDbGkJah

* **Artista:** NF
* **Título:** HOPE
* **Álbum:** HOPE
* **Link:** https://open.spotify.com/track/0EgLxY52mpGsXETyEsgVlP

* **Artista:** Wiz Khalifa
* **Título:** Hopeless Romantic (feat. Swae Lee)
* **Álbum:** Rolling Papers 2
* **Link:** https://open.spotify.com/track/7IBSffWIu7P2MC7kMwy2FM

* **Artista:** Arctic Monkeys
* **Título:** 505
* **Álbum:** Favourite Worst Nightmare (Standard Version)
* **Link:** https://open.spotify.com/track/58ge6dfP91o9oXMzq3XkIS

* **Artista:** Fleetwood Mac
* **Título:** Dreams - 2004 Remaster
* **Álbum:** Rumours (Super Deluxe)
* **Link:** https://open.spotify.com/track/0ofHAoxe9vBkTCp2UQIavz

* **Artista:** The Goo Goo Dolls
* **Título:** Iris
* **Álbum:** Dizzy up the Girl
* **Link:** https://open.spotify.com/track/6Qyc6fS4DsZjB2mRW